In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# colab only (add keys using the key icon on the left)
# from google.colab import userdata
# apikey = userdata.get('OPEN_WEATHER_API_KEY')

In [3]:
############# DO NOT DO THIS ################
# store your API key(s) in environment variables
apikey = os.environ['OPEN_WEATHER_API_KEY']

**Documentation:** <br>
https://openweathermap.org/api/geocoding-api <br>
https://openweathermap.org/price

## Basic Response from API

In [4]:
# Example 1: get lattitude and longitude of a city
cityname = 'Almaty'
URL = f'http://api.openweathermap.org/geo/1.0/direct?q={cityname}&limit=1&appid={apikey}'
response = requests.get(URL)
response

<Response [200]>

In [ ]:
response.json()

[{'name': 'Almaty',
  'local_names': {'eo': 'Almato',
   'gl': 'Almati',
   'ar': 'ألماتي',
   'br': 'Almaty',
   'bg': 'Алмати',
   'ps': 'الماته',
   'ka': 'ალმათი',
   'feature_name': 'Almaty',
   'os': 'Алматы',
   'fi': 'Almaty',
   'it': 'Almaty',
   'lv': 'Almati',
   'ko': '알마티',
   'sq': 'Almaty',
   'no': 'Almaty',
   'zh': '阿拉木圖',
   'es': 'Almatý',
   'af': 'Almati',
   'ascii': 'Almaty',
   'ta': 'அல்மாத்தி',
   'mr': 'अल्माटी',
   'ro': 'Almatî',
   'ba': 'Алматы',
   'la': 'Almata',
   'az': 'Almatı',
   'kn': 'ಅಲ್ಮಾಟೀ',
   'mk': 'Алмати',
   'am': 'አልማቲ',
   'sh': 'Almati',
   'de': 'Almaty',
   'hy': 'Ալմաթի',
   'io': 'Almaty',
   'cv': 'Алматы',
   'tt': 'Алматы',
   'hu': 'Almati',
   'ja': 'アルマトイ',
   'nl': 'Almaty',
   'pl': 'Ałmaty',
   'en': 'Almaty',
   'hi': 'अलमाती',
   'yi': 'אלמאטיע',
   'th': 'อัลมาเตอ',
   'gd': 'Almaty',
   'ru': 'Алматы',
   'ga': 'Almaty',
   'tr': 'Almatı',
   'da': 'Almaty',
   'bn': 'আলমাতি',
   'nn': 'Almaty',
   'id': 'Almaty',
  

In [5]:
# get lat and lon from the json
lat, lon = response.json()[0]['lat'], response.json()[0]['lon']

In [6]:
lat, lon

(43.2363924, 76.9457275)

## More complex JSON parsing

In [12]:
# Example 2: get a 5 day weather forecast
URL = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={apikey}'
response = requests.get(URL)
response

<Response [200]>

In [13]:
response.text[:100]

'{"cod":"200","message":0,"cnt":40,"list":[{"dt":1752786000,"main":{"temp":297.81,"feels_like":297.22'

In [14]:
response.json().keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [17]:
response.json()["city"]

{'id': 1526384,
 'name': 'Almaty',
 'coord': {'lat': 43.2364, 'lon': 76.9457},
 'country': 'KZ',
 'population': 2000900,
 'timezone': 18000,
 'sunrise': 1752708438,
 'sunset': 1752762540}

In [22]:
info = response.json()["list"]
info

[{'dt': 1752786000,
  'main': {'temp': 297.81,
   'feels_like': 297.22,
   'temp_min': 297.79,
   'temp_max': 297.81,
   'pressure': 1010,
   'sea_level': 1010,
   'grnd_level': 870,
   'humidity': 34,
   'temp_kf': 0.02},
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03n'}],
  'clouds': {'all': 27},
  'wind': {'speed': 3.71, 'deg': 156, 'gust': 2.86},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2025-07-17 21:00:00'},
 {'dt': 1752796800,
  'main': {'temp': 297.65,
   'feels_like': 296.97,
   'temp_min': 297.56,
   'temp_max': 297.65,
   'pressure': 1011,
   'sea_level': 1011,
   'grnd_level': 870,
   'humidity': 31,
   'temp_kf': 0.09},
  'weather': [{'id': 801,
    'main': 'Clouds',
    'description': 'few clouds',
    'icon': '02d'}],
  'clouds': {'all': 13},
  'wind': {'speed': 3.18, 'deg': 164, 'gust': 2.54},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2025-07-18 00:00:00'},


In [ ]:
# parse JSON into a DataFrame containing temperature, humidity and wind speed
times = [pd.to_datetime(t['dt_txt']) for t in info]
temps = [float(t['main']['temp']) for t in info]
humidities = [int(h['main']['humidity']) for h in info]
wind_speeds = [float(w['wind']['speed']) for w in info]

forecast_df = pd.DataFrame({'time':times,
							'temperature':temps,
							'humidity':humidities,
							'wind_speed':wind_speeds})
forecast_df.head()

,time,temperature,humidity,wind_speed
0,2025-07-17 21:00:00,297.81,34,3.71
1,2025-07-18 00:00:00,297.65,31,3.18
2,2025-07-18 03:00:00,303.81,21,2.39
3,2025-07-18 06:00:00,308.15,18,2.66
4,2025-07-18 09:00:00,310.81,15,3.52


## Using headers and parameters

In [ ]:
# Example 3: get 5 day weather forecast with metric units, in French, with 10 timestamps and in XML mode
params_dict = {
			   'lat' : lat,
			   'lon' : lon,
			   'units' : 'metric',
			   'lang' : 'fr',
			   'cnt' : '10',
			   'mode' : 'xml',
			   'apikey' : apikey
			  }

headers_dict = {'accept': 'application/XML'}
URL = 'http://api.openweathermap.org/data/2.5/forecast'
response = requests.get(URL, headers=headers_dict, params=params_dict)
response

<Response [200]>

In [24]:
response.text

'<?xml version="1.0" encoding="UTF-8"?>\n<weatherdata><location><name>Almaty</name><type></type><country>KZ</country><timezone>18000</timezone><location altitude="0" latitude="43.2364" longitude="76.9457" geobase="geonames" geobaseid="1526384"></location></location><credit></credit><meta><lastupdate></lastupdate><calctime>0</calctime><nextupdate></nextupdate></meta><sun rise="2025-07-16T23:27:18" set="2025-07-17T14:29:00"></sun><forecast><time from="2025-07-17T18:00:00" to="2025-07-17T21:00:00"><symbol number="802" name="partiellement nuageux" var="03n"></symbol><precipitation probability="0"></precipitation><windDirection deg="156" code="SSE" name="South-southeast"></windDirection><windSpeed mps="3.71" unit="m/s" name="Gentle Breeze"></windSpeed><windGust gust="2.86" unit="m/s"></windGust><temperature unit="celsius" value="24.66" min="24.64" max="24.66"></temperature><feels_like value="24.07" unit="celsius"></feels_like><pressure unit="hPa" value="1010"></pressure><humidity value="34"

## Parsing XML

`pip install lxml` <br>
`conda install lxml`


In [25]:
soup = BeautifulSoup(response.text, 'lxml')
soup

C:\Users\sjmit\AppData\Local\Temp\ipykernel_27128\4254831159.py:1: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, 'lxml')


<?xml version="1.0" encoding="UTF-8"?><html><body><weatherdata><location><name>Almaty</name><type></type><country>KZ</country><timezone>18000</timezone><location altitude="0" geobase="geonames" geobaseid="1526384" latitude="43.2364" longitude="76.9457"></location></location><credit></credit><meta/><lastupdate></lastupdate><calctime>0</calctime><nextupdate></nextupdate><sun rise="2025-07-16T23:27:18" set="2025-07-17T14:29:00"></sun><forecast><time from="2025-07-17T18:00:00" to="2025-07-17T21:00:00"><symbol name="partiellement nuageux" number="802" var="03n"></symbol><precipitation probability="0"></precipitation><winddirection code="SSE" deg="156" name="South-southeast"></winddirection><windspeed mps="3.71" name="Gentle Breeze" unit="m/s"></windspeed><windgust gust="2.86" unit="m/s"></windgust><temperature max="24.66" min="24.64" unit="celsius" value="24.66"></temperature><feels_like unit="celsius" value="24.07"></feels_like><pressure unit="hPa" value="1010"></pressure><humidity unit="%

In [26]:
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <weatherdata>
   <location>
    <name>
     Almaty
    </name>
    <type>
    </type>
    <country>
     KZ
    </country>
    <timezone>
     18000
    </timezone>
    <location altitude="0" geobase="geonames" geobaseid="1526384" latitude="43.2364" longitude="76.9457">
    </location>
   </location>
   <credit>
   </credit>
   <meta/>
   <lastupdate>
   </lastupdate>
   <calctime>
    0
   </calctime>
   <nextupdate>
   </nextupdate>
   <sun rise="2025-07-16T23:27:18" set="2025-07-17T14:29:00">
   </sun>
   <forecast>
    <time from="2025-07-17T18:00:00" to="2025-07-17T21:00:00">
     <symbol name="partiellement nuageux" number="802" var="03n">
     </symbol>
     <precipitation probability="0">
     </precipitation>
     <winddirection code="SSE" deg="156" name="South-southeast">
     </winddirection>
     <windspeed mps="3.71" name="Gentle Breeze" unit="m/s">
     </windspeed>
     <windgust gust="2.86" unit="m/s">
     </windg

In [28]:
# extract country code
soup.find('country').text

'KZ'

In [30]:
soup.find_all('temperature')[0]["min"]

'24.64'

In [31]:
soup.find_all('temperature')

[<temperature max="24.66" min="24.64" unit="celsius" value="24.66"></temperature>,
 <temperature max="24.5" min="24.41" unit="celsius" value="24.5"></temperature>,
 <temperature max="30.66" min="30.66" unit="celsius" value="30.66"></temperature>,
 <temperature max="35" min="35" unit="celsius" value="35"></temperature>,
 <temperature max="37.66" min="37.66" unit="celsius" value="37.66"></temperature>,
 <temperature max="36.93" min="36.93" unit="celsius" value="36.93"></temperature>,
 <temperature max="28.27" min="28.27" unit="celsius" value="28.27"></temperature>,
 <temperature max="27.12" min="27.12" unit="celsius" value="27.12"></temperature>,
 <temperature max="25.6" min="25.6" unit="celsius" value="25.6"></temperature>,
 <temperature max="25.34" min="25.34" unit="celsius" value="25.34"></temperature>]

In [ ]:
# get all temperature ranges
[float(temp['max']) - float(temp['min']) for temp in soup.find_all('temperature')]

[1.7699999999999996,
 4.939999999999998,
 4.200000000000003,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
# parse XML into a DataFrame containing temperature, humidity and wind speed
times = [pd.to_datetime(t['from']) for t in soup.find_all('time')]
temps = [float(t['value']) for t in soup.find_all('temperature')]
humidities = [int(h['value']) for h in soup.find_all('humidity')]
wind_speeds = [float(w['mps']) for w in soup.find_all('windspeed')]

forecast_df = pd.DataFrame({'time':times,
							'temp':temps,
							'humidity':humidities,
							'wind_speed':wind_speeds})
forecast_df.head()

,time,temp,humidity,wind_speed
0,2025-07-17 18:00:00,24.66,34,3.71
1,2025-07-17 21:00:00,24.50,31,3.18
2,2025-07-18 00:00:00,30.66,21,2.39
3,2025-07-18 03:00:00,35.00,18,2.66
4,2025-07-18 06:00:00,37.66,15,3.52


## JSON parsing exercise

In [ ]:
#@title example JSON about the Office TV show with nested structure
the_office_json = {
	"title": "The Office",
	"characters": [
		{
			"Michael Scott": {
				"position": "Regional Manager",
				"hobbies": ["ice hockey", "magic tricks", "improv"],
				"seasons": [1, 2, 3, 4, 5, 6, 7],
				"catchphrase": "That's what she said!",
				"relationships": ["Holly Flax", "Jan Levinson"]
			}
		},
		{
			"Jim Halpert": {
				"position": "Sales Representative",
				"hobbies": ["pranking Dwight", "basketball", "cycling"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "I am about to do something very bold at this job that I've never done before. Try.",
				"relationships": ["Pam Beesly"]
			}
		},
		{
			"Dwight Schrute": {
				"position": "Assistant to the Regional Manager",
				"hobbies": ["beet farming", "martial arts", "survivalism"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "False.",
				"relationships": ["Angela Martin"]
			}
		},
		{
			"Pam Beesly": {
				"position": "Receptionist",
				"hobbies": ["art", "volleyball"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Dunder Mifflin, this is Pam.",
				"relationships": ["Jim Halpert"]
			}
		},
		{
			"Kelly Kapoor": {
				"position": "Customer Service Representative",
				"hobbies": ["fashion", "gossip", "social media"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "This day is bananas, B-A-N-A-N-A-S!",
				"relationships": ["Ryan Howard"]
			}
		},
		{
			"Angela Martin": {
				"position": "Head of Accounting",
				"hobbies": ["cats", "conservative Christianity"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "I don't have a headache. I'm just preparing.",
				"relationships": ["Dwight Schrute", "Andy Bernard"]
			}
		},
		{
			"Kevin Malone": {
				"position": "Accountant",
				"hobbies": ["gambling", "cooking chili"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "I do the numbers.",
				"relationships": []
			}
		},
		{
			"Oscar Martinez": {
				"position": "Accountant",
				"hobbies": ["cycling", "film"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Actually...",
				"relationships": []
			}
		},
		{
			"Phyllis Vance": {
				"position": "Sales Representative",
				"hobbies": ["knitting", "bird-watching"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Close your mouth, sweetie. You look like a trout.",
				"relationships": ["Bob Vance (Vance Refrigeration)"]
			}
		},
		{
			"Stanley Hudson": {
				"position": "Sales Representative",
				"hobbies": ["crossword puzzles", "pretzel day"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Did I stutter?",
				"relationships": []
			}
		},
		{
			"Darryl Philbin": {
				"position": "Warehouse Foreman",
				"hobbies": ["chess", "keyboard"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Dinkin' flicka.",
				"relationships": []
			}
		},
		{
			"Creed Bratton": {
				"position": "Quality Assurance",
				"hobbies": ["scuba diving", "rock music"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "If I can't scuba, then what's this all been about? What am I working toward?",
				"relationships": []
			}
		},
		{
			"Meredith Palmer": {
				"position": "Supplier Relations",
				"hobbies": ["partying", "alcohol"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "I've done some things I'm not proud of.",
				"relationships": []
			}
		},
		{
			"Ryan Howard": {
				"position": "Temp",
				"hobbies": ["business school", "social media startups"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "I'm not saying I had a meteoric rise, but I did.",
				"relationships": ["Kelly Kapoor"]
			}
		},
		{
			"Toby Flenderson": {
				"position": "Human Resources Representative",
				"hobbies": ["writing", "running"],
				"seasons": [1, 2, 3, 4, 5, 6, 7, 8, 9],
				"catchphrase": "Does anyone have a camera here?",
				"relationships": []
			}
		},
		{
			"Holly Flax": {
				"position": "Human Resources Representative",
				"hobbies": ["yoga", "improv"],
				"seasons": [4, 5, 7],
				"catchphrase": "Much to learn, you still have. Love, the best thing is.",
				"relationships": ["Michael Scott"]
			}
		}
	],
	"seasons": [
		{
			"Season 1": {
				"year": 2005,
				"episodes": [
					{
						"title": "Pilot",
						"director": "Ken Kwapis",
						"air_date": "2005-03-24"
					},
					{
						"title": "Diversity Day",
						"director": "Ken Kwapis",
						"air_date": "2005-03-29"
					}
				]
			}
		},
		{
			"Season 2": {
				"year": 2005,
				"episodes": [
					{
						"title": "The Dundies",
						"director": "Greg Daniels",
						"air_date": "2005-09-20"
					},
					{
						"title": "Sexual Harassment",
						"director": "Ken Kwapis",
						"air_date": "2005-09-27"
					}
				]
			}
		}
	],
	"locations": {
		"primary": "Scranton, Pennsylvania",
		"branch": "Dunder Mifflin Paper Company",
		"address": "1725 Slough Avenue"
	},
	"awards": [
		{
			"name": "Primetime Emmy Award",
			"category": "Outstanding Comedy Series",
			"year": 2006
		},
		{
			"name": "Golden Globe Award",
			"category": "Best Actor - Television Series Musical or Comedy",
			"recipient": "Steve Carell",
			"year": 2006
		}
	]
}

In [70]:
list_of_chars

[{'Michael Scott': {'position': 'Regional Manager',
   'hobbies': ['ice hockey', 'magic tricks', 'improv'],
   'seasons': [1, 2, 3, 4, 5, 6, 7],
   'catchphrase': "That's what she said!",
   'relationships': ['Holly Flax', 'Jan Levinson']}},
 {'Jim Halpert': {'position': 'Sales Representative',
   'hobbies': ['pranking Dwight', 'basketball', 'cycling'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': "I am about to do something very bold at this job that I've never done before. Try.",
   'relationships': ['Pam Beesly']}},
 {'Dwight Schrute': {'position': 'Assistant to the Regional Manager',
   'hobbies': ['beet farming', 'martial arts', 'survivalism'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': 'False.',
   'relationships': ['Angela Martin']}},
 {'Pam Beesly': {'position': 'Receptionist',
   'hobbies': ['art', 'volleyball'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': 'Dunder Mifflin, this is Pam.',
   'relationships': ['Jim Halpert']}},


In [72]:
lst = [4, 5, 7]
lst = [str(x) for x in lst]
", ".join(lst)

'4, 5, 7'

In [78]:
lst = [1]
lst.append([2, 3, 4])
lst

[1, [2, 3, 4]]

In [95]:
data_lst = []
for char in list_of_chars:
	name = list(char.keys())[0]
	char_dict = char[name]
	columns = list(char_dict.keys())
	values = char_dict.values()
	
	row_lst = [name]
	for val in values:
		if type(val) == str:
			row_lst.append(val)
		else:
			val = [str(x) for x in val]
			row_lst.append(", ".join(val))

	data_lst.append(row_lst)

df = pd.DataFrame(data=data_lst, columns=["character"]+columns)
df

,character,position,hobbies,seasons,catchphrase,relationships
0,Michael Scott,Regional Manager,"ice hockey, magic tricks, improv","1, 2, 3, 4, 5, 6, 7",That's what she said!,"Holly Flax, Jan Levinson"
1,Jim Halpert,Sales Representative,"pranking Dwight, basketball, cycling","1, 2, 3, 4, 5, 6, 7, 8, 9",I am about to do something very bold at this j...,Pam Beesly
2,Dwight Schrute,Assistant to the Regional Manager,"beet farming, martial arts, survivalism","1, 2, 3, 4, 5, 6, 7, 8, 9",False.,Angela Martin
3,Pam Beesly,Receptionist,"art, volleyball","1, 2, 3, 4, 5, 6, 7, 8, 9","Dunder Mifflin, this is Pam.",Jim Halpert
4,Kelly Kapoor,Customer Service Representative,"fashion, gossip, social media","1, 2, 3, 4, 5, 6, 7, 8, 9","This day is bananas, B-A-N-A-N-A-S!",Ryan Howard
5,Angela Martin,Head of Accounting,"cats, conservative Christianity","1, 2, 3, 4, 5, 6, 7, 8, 9",I don't have a headache. I'm just preparing.,"Dwight Schrute, Andy Bernard"
6,Kevin Malone,Accountant,"gambling, cooking chili","1, 2, 3, 4, 5, 6, 7, 8, 9",I do the numbers.,
7,Oscar Martinez,Accountant,"cycling, film","1, 2, 3, 4, 5, 6, 7, 8, 9",Actually...,
8,Phyllis Vance,Sales Representative,"knitting, bird-watching","1, 2, 3, 4, 5, 6, 7, 8, 9","Close your mouth, sweetie. You look like a trout.",Bob Vance (Vance Refrigeration)
9,Stanley Hudson,Sales Representative,"crossword puzzles, pretzel day","1, 2, 3, 4, 5, 6, 7, 8, 9",Did I stutter?,


In [35]:
list_of_chars = the_office_json["characters"]
list_of_chars

[{'Michael Scott': {'position': 'Regional Manager',
   'hobbies': ['ice hockey', 'magic tricks', 'improv'],
   'seasons': [1, 2, 3, 4, 5, 6, 7],
   'catchphrase': "That's what she said!",
   'relationships': ['Holly Flax', 'Jan Levinson']}},
 {'Jim Halpert': {'position': 'Sales Representative',
   'hobbies': ['pranking Dwight', 'basketball', 'cycling'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': "I am about to do something very bold at this job that I've never done before. Try.",
   'relationships': ['Pam Beesly']}},
 {'Dwight Schrute': {'position': 'Assistant to the Regional Manager',
   'hobbies': ['beet farming', 'martial arts', 'survivalism'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': 'False.',
   'relationships': ['Angela Martin']}},
 {'Pam Beesly': {'position': 'Receptionist',
   'hobbies': ['art', 'volleyball'],
   'seasons': [1, 2, 3, 4, 5, 6, 7, 8, 9],
   'catchphrase': 'Dunder Mifflin, this is Pam.',
   'relationships': ['Jim Halpert']}},


In [94]:
# parse the JSON into a dataframe to have all character information
characters_json = the_office_json['characters']
characters = []

for character in characters_json:

	name, data = list(character.items())[0]
	position = data['position']
	hobbies = ', '.join(data['hobbies'])
	seasons = ', '.join(map(str, data['seasons']))
	catchphrase = data['catchphrase']
	relationships = ', '.join(data['relationships'])
	characters.append([name, position, hobbies, catchphrase, relationships, seasons])


pd.DataFrame(characters, columns=['name', 'position', 'hobbies', 'catchphrase', 'relationships', 'seasons'])

,name,position,hobbies,catchphrase,relationships,seasons
0,Michael Scott,Regional Manager,"ice hockey, magic tricks, improv",That's what she said!,"Holly Flax, Jan Levinson","1, 2, 3, 4, 5, 6, 7"
1,Jim Halpert,Sales Representative,"pranking Dwight, basketball, cycling",I am about to do something very bold at this j...,Pam Beesly,"1, 2, 3, 4, 5, 6, 7, 8, 9"
2,Dwight Schrute,Assistant to the Regional Manager,"beet farming, martial arts, survivalism",False.,Angela Martin,"1, 2, 3, 4, 5, 6, 7, 8, 9"
3,Pam Beesly,Receptionist,"art, volleyball","Dunder Mifflin, this is Pam.",Jim Halpert,"1, 2, 3, 4, 5, 6, 7, 8, 9"
4,Kelly Kapoor,Customer Service Representative,"fashion, gossip, social media","This day is bananas, B-A-N-A-N-A-S!",Ryan Howard,"1, 2, 3, 4, 5, 6, 7, 8, 9"
5,Angela Martin,Head of Accounting,"cats, conservative Christianity",I don't have a headache. I'm just preparing.,"Dwight Schrute, Andy Bernard","1, 2, 3, 4, 5, 6, 7, 8, 9"
6,Kevin Malone,Accountant,"gambling, cooking chili",I do the numbers.,,"1, 2, 3, 4, 5, 6, 7, 8, 9"
7,Oscar Martinez,Accountant,"cycling, film",Actually...,,"1, 2, 3, 4, 5, 6, 7, 8, 9"
8,Phyllis Vance,Sales Representative,"knitting, bird-watching","Close your mouth, sweetie. You look like a trout.",Bob Vance (Vance Refrigeration),"1, 2, 3, 4, 5, 6, 7, 8, 9"
9,Stanley Hudson,Sales Representative,"crossword puzzles, pretzel day",Did I stutter?,,"1, 2, 3, 4, 5, 6, 7, 8, 9"


In [ ]:
#@title possible answer
characters = []

for character in the_office_json['characters']:
	# print(character)
	name, data = list(character.items())[0]
	# print(name, data)
	position = data['position']
	hobbies = ', '.join(data['hobbies'])
	seasons = ', '.join(map(str, data['seasons']))
	catchphrase = data['catchphrase']
	relationships = ', '.join(data['relationships'])
	characters.append([name, position, hobbies, catchphrase, relationships, seasons])

answer = pd.DataFrame(characters, columns=['name', 'position', 'hobbies',
																					 'catchphrase', 'relationships', 'seasons'])